# Tasks And Data

- Refer to example_data

In [ ]:
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class BatchElement:
    """Abstract base class for BatchElements. Store all kinds of data being passed around CHEESE"""
    pass

@dataclass
class TextCaptionBatchElement(BatchElement):
    id : int
    text : str # Text we want captioner to see
    caption_index : List[Tuple[int, int]] # List of tuples of (start, end) indice
    captions : List[str]
        
@dataclass
class Task:
    """
    Task for orchestrator to execute.
    """
    data : BatchElement
    client_id : int = -1 # Once a task has been assigned to client, want to keep track
    model_id : int = -1 # Once a task has been assigned to model, want to keep track
    # For simplicities sake, let -2 denote we are looking for a model
    
example_data = TextCaptionBatchElement(
    id = 1,
    text = "The goose waddled into the coffee shop and looked around. It was a bright and cheery place, with warm colors and a comfortable feeling. The goose liked it immediately. It waddled up to the counter and peered at the menu. There were all sorts of delicious-looking drinks, but the goose didn\'t know what any of them were. It honked in frustration. The barista, a young woman with a kind face, came over. \"Can I help you?\" she asked. The goose honked again and pointed at the menu. The barista smiled. \"Let me guess, you want a cappuccino?\" The goose nodded eagerly. \"Coming right up!\" The barista got to work, and soon the goose had a steaming cup of cappuccino in front of it. It sipped cautiously at first, but then it started gulping it down. It was delicious! The goose finished its drink and waddled over to a cozy armchair by the window. It settled down with a contented sigh and watched the people go by outside. This was the life.",
    caption_index = [],
    captions = []
)

example_task = Task(
    example_data
)

# Backend Point of Communciation

This was written under assumption of using gradio (gr), so there may be several methods or attributes that can be ignored for our use case. I can update object later as needed. The most important methods are update, complete_task and launch. The most important attribute is data.   
  
- ClientFront.update(data) will be used by the backend to update the data we wish to show client
- After user has submitted their labels/captions, ClientFront.data should be updated to reflect user submission, and ClientFront.complete_task() should be called after this variable has been set

- Note that in current launch method, a demo object is used which should be defined in a subclass to ClientFront (i.e. which frontend specifically to use is defined in subclass depending on type of task)


In [ ]:
from backend.client import ClientManager, Client
import gradio as gr

class ClientFront:
    def __init__(self):
        self.demo : gr.Interface = None
        self.data : BatchElement = None # Data currently being shown to user
        self.buffer : BatchElement = None # Buffer for next data to show user

        self.showing_data = False # is data visible to user currently?
        self.client : Client = None

        self.launched = False

    def set_client(self, parent : Client):
        """
        Set client that is "parent" to this frontend.
        :param parent: Parent of frontend
        :type parent: Client
        """
        self.client = parent
    
    def update(self, data : BatchElement):
        """
        Update the buffer with new data
        """
        self.buffer = data

    def complete_task(self):
        """
        Finish task and wipe data. Ping parent client
        """
        self.showing_data = False
        self.client.front_ping()
        self.data = None
    
    def refresh(self) -> bool:
        """
        Refresh data from buffer.
        :return: Whether or not there is new data to present after the refresh
        :rtype: bool
        """
        if self.buffer is not None:
            self.data = self.buffer
            self.buffer = None
        if self.data is not None:
            self.showing_data = True
            return True
        return False

    def launch(self) -> str:
        """
        Launch the frontend application and return URL to access it
        :return: URL for user to access frontend
        :rtype: str
        """
        try:
            assert self.client is not None
        except:
            raise Exception("Error: Launched frontend with unspecified parent client")

        _, _, url = self.demo.launch(
            share = True, quiet = True,
            prevent_thread_lock = True,
        )
        self.launched = True
        return url